In [1]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import logging
from langchain.chains import LLMChain
import os
import time
from dotenv import load_dotenv

load_dotenv()

# Configuración del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def tiempo_de_ejecucion(func):
    """
    Decorador que registra el tiempo de ejecución de una función.
    """
    def wrapper(*args, **kwargs):
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fin = time.time()
        logging.info(f"{func.__name__} tomó {fin - inicio} segundos en ejecutarse.")
        return resultado
    return wrapper

In [2]:
# Example usage

OpenAI.api_key = os.getenv("OPENAI_API_KEY")
json_path = '/Users/adrianinfantes/Desktop/AIR/CollegeStudies/MachineLearningPath/Portfolio/DevProjects/TriviaFlix/data/PreguntasYRespuestas.json'

In [3]:
class MiProcesadorNLP:
    def __init__(self, temperatura=0.5, max_tokens=100, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0):
        self.template = """{question}"""
        self.prompt = PromptTemplate(input_variables=["question"], template=self.template)
        self.llm = OpenAI(temperature=temperatura, max_tokens=max_tokens, top_p=top_p, frequency_penalty=frequency_penalty, presence_penalty=presence_penalty)
        self.chain = LLMChain(llm=self.llm, prompt=self.prompt)
        
    @tiempo_de_ejecucion
    def ejecutar(self, question):
        return self.chain.run(question=question)

In [4]:
procesador = MiProcesadorNLP()
resultado = procesador.ejecutar(question="¿Cuál es la capital de Francia?")
logging.info(f"Resultado: {resultado}")

/Users/adrianinfantes/Library/Caches/pypoetry/virtualenvs/triviatflix-C5VKzA79-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/adrianinfantes/Library/Caches/pypoetry/virtualenvs/triviatflix-C5VKzA79-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/Users/adrianinfantes/Library/Caches/pypoetry/virtualenvs/triviatflix-C5VKzA79-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecat

In [5]:
import json

# Configuración del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class ProcesadorJson:
    def __init__(self, ruta_archivo):
        self.ruta_archivo = ruta_archivo

    def cargar_json(self):
        """
        Carga un archivo JSON y devuelve su contenido.
        """
        try:
            with open(self.ruta_archivo, "r") as f:
                data = json.load(f)
            logging.info(f"Archivo JSON cargado con éxito: '{self.ruta_archivo}'.")
            return data
        except Exception as e:
            logging.error(f"Error al cargar el archivo JSON: {e}")
            return None

    def preparar_texto(self):
        """
        Extrae títulos y opciones de las preguntas del JSON, concatenándolos en una única cadena de texto.
        """
        data = self.cargar_json()
        if data:
            texto_completo = ' '.join([
                f"{item['titulo']} Opción A: {item['opcionA']} Opción B: {item['opcionB']} " +
                f"Opción C: {item['opcionC']} Opción D: {item['opcionD']}" 
                for item in data
            ])
            logging.info("Texto preparado con éxito a partir de JSON.")
            return texto_completo
        else:
            logging.error("No se pudo cargar datos para preparar texto.")
            return ""

In [6]:
# Uso de ProcesadorJson para cargar un archivo JSON

procesador_json = ProcesadorJson(ruta_archivo=json_path)
texto_completo = procesador_json.preparar_texto()
print(texto_completo)

2024-05-11 22:57:05,272 - INFO - Archivo JSON cargado con éxito: '/Users/adrianinfantes/Desktop/AIR/CollegeStudies/MachineLearningPath/Portfolio/DevProjects/TriviaFlix/data/PreguntasYRespuestas.json'.
2024-05-11 22:57:05,273 - INFO - Texto preparado con éxito a partir de JSON.


¿Cuál es el nombre del personaje principal en la serie 'Stranger Things'? Opción A: Mike Wheeler Opción B: Eleven Opción C: Dustin Henderson Opción D: Will Byers ¿Quién interpreta a Frank Underwood en la serie 'House of Cards'? Opción A: Kevin Spacey Opción B: Robin Wright Opción C: Michael Kelly Opción D: Kate Mara ¿Qué serie documental de crímenes reales se hizo famosa por su episodio titulado 'Making a Murderer'? Opción A: The Keepers Opción B: The Jinx Opción C: Making a Murderer Opción D: The Staircase ¿Cuál es el título de la serie sobre el nacimiento de la música hip-hop en Nueva York en los años 70 y 80? Opción A: The Get Down Opción B: Hip-Hop Evolution Opción C: Rhythm + Flow Opción D: Empire ¿Quién es el creador de la serie 'Black Mirror'? Opción A: Charlie Brooker Opción B: Dan Harmon Opción C: David Benioff Opción D: D.B. Weiss ¿Cuál es el título de la película sobre la vida de Barack Obama, protagonizada por Devon Terrell? Opción A: Barry Opción B: The Obama Effect Opción

In [7]:
from langchain.text_splitter import CharacterTextSplitter
import logging

class SeparadorTexto:
    def __init__(self, separator=".", chunk_size=1000, chunk_overlap=200):
        self.text_splitter = CharacterTextSplitter(
            separator=separator,
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
        )
    
    def dividir_texto(self, texto):
        """
        Divide el texto en segmentos más pequeños basados en la configuración inicial.
        """
        try:
            texts = self.text_splitter.split_text(texto)
            logging.info(f"Texto dividido en {len(texts)} segmentos.")
            return texts
        except Exception as e:
            logging.error(f"Error al dividir texto: {e}")
            return []

In [8]:
# Integración con el flujo de trabajo

if texto_completo:
    separador = SeparadorTexto()
    segmentos_texto = separador.dividir_texto(texto_completo)
    
    if segmentos_texto:
        # Procesar cada segmento con `MiProcesadorNLP` u otra lógica específica
        # Por ejemplo, imprimir el primer segmento
        print(segmentos_texto[0])
    else:
        print("No se pudo dividir el texto.")
else:
    print("No se pudo extraer texto del archivo JSON.")

2024-05-11 22:57:07,780 - WARNING - Created a chunk of size 3162, which is longer than the specified 1000
2024-05-11 22:57:07,781 - WARNING - Created a chunk of size 1194, which is longer than the specified 1000
2024-05-11 22:57:07,781 - INFO - Texto dividido en 4 segmentos.


¿Cuál es el nombre del personaje principal en la serie 'Stranger Things'? Opción A: Mike Wheeler Opción B: Eleven Opción C: Dustin Henderson Opción D: Will Byers ¿Quién interpreta a Frank Underwood en la serie 'House of Cards'? Opción A: Kevin Spacey Opción B: Robin Wright Opción C: Michael Kelly Opción D: Kate Mara ¿Qué serie documental de crímenes reales se hizo famosa por su episodio titulado 'Making a Murderer'? Opción A: The Keepers Opción B: The Jinx Opción C: Making a Murderer Opción D: The Staircase ¿Cuál es el título de la serie sobre el nacimiento de la música hip-hop en Nueva York en los años 70 y 80? Opción A: The Get Down Opción B: Hip-Hop Evolution Opción C: Rhythm + Flow Opción D: Empire ¿Quién es el creador de la serie 'Black Mirror'? Opción A: Charlie Brooker Opción B: Dan Harmon Opción C: David Benioff Opción D: D.B


In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import logging

class BuscadorDocumentos:
    def __init__(self):
        self.embeddings = OpenAIEmbeddings()
        self.docsearch = None  # Se inicializará después con los textos
    
    def inicializar_busqueda(self, texts):
        """
        Inicializa el motor de búsqueda FAISS con los textos proporcionados.
        """
        try:
            self.docsearch = FAISS.from_texts(texts, self.embeddings)
            logging.info("Motor de búsqueda FAISS inicializado con éxito.")
        except Exception as e:
            logging.error(f"Error al inicializar FAISS: {e}")
            self.docsearch = None
    
    def buscar(self, query, top_n=5, search_type='similarity'):
        """
        Realiza una búsqueda en el motor FAISS y devuelve los top_n resultados.
        Se espera que search_type sea 'similarity' o 'mmr'.
        """
        if not self.docsearch:
            logging.error("Motor de búsqueda FAISS no inicializado.")
            return []
        try:
            resultados = self.docsearch.search(query, top_n=top_n, search_type=search_type)
            logging.info(f"Búsqueda realizada con éxito. {len(resultados)} resultados encontrados.")
            return resultados
        except Exception as e:
            logging.error(f"Error durante la búsqueda: {e}")
            return []

In [10]:
buscador = BuscadorDocumentos()
buscador.inicializar_busqueda(segmentos_texto)

# Realizar una búsqueda de ejemplo
resultados_busqueda = buscador.buscar("Consulta de ejemplo", top_n=3)
if resultados_busqueda:
    print("Resultados de la búsqueda:", resultados_busqueda)
else:
    print("No se encontraron resultados.")

/Users/adrianinfantes/Library/Caches/pypoetry/virtualenvs/triviatflix-C5VKzA79-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-05-11 22:57:13,099 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-11 22:57:13,224 - INFO - Loading faiss.
2024-05-11 22:57:13,250 - INFO - Successfully loaded faiss.
2024-05-11 22:57:13,254 - INFO - Motor de búsqueda FAISS inicializado con éxito.
2024-05-11 22:57:13,443 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-11 22:57:13,450 - INFO - Búsqueda realizada con éxito. 4 resultados encontrados.


Resultados de la búsqueda: [Document(page_content="Maisel ¿Cuál es el título de la película de ciencia ficción protagonizada por Natalie Portman, donde un equipo de mujeres investiga una misteriosa área conocida como 'La Zona'? Opción A: Annihilation Opción B: Ex Machina Opción C: Arrival Opción D: Interstellar ¿En qué película original de Netflix un joven se infiltra en una pandilla de carretera en Los Ángeles para investigar la desaparición de su hermano? Opción A: Imperial Dreams Opción B: Beasts of No Nation Opción C: Mudbound Opción D: Lost Bullet ¿Cuál es el título de la película de terror sobrenatural donde una familia debe sobrevivir a una serie de fenómenos paranormales en su nueva casa? Opción A: The Conjuring Opción B: Sinister Opción C: Insidious Opción D: The Haunting of Hill House ¿Quién dirigió la película 'El Ritual', donde un grupo de amigos enfrenta a una entidad sobrenatural en un bosque sueco? Opción A: Ari Aster Opción B: Mike Flanagan Opción C: David Bruckner Opci

In [11]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI


class ProcesadorQA:
    def __init__(self, buscador_documentos):
        self.buscador_documentos = buscador_documentos
        self.chain = load_qa_chain(OpenAI(), chain_type="stuff")
    
    def ejecutar_qa(self, query):
        """
        Ejecuta una consulta de preguntas y respuestas utilizando documentos
        relevantes obtenidos a través de una búsqueda de similitud.
        """
        try:
            # Realiza una búsqueda de similitud para encontrar documentos relevantes
            docs = self.buscador_documentos.docsearch.similarity_search(query)
            # Ejecuta la cadena QA con la pregunta y los documentos encontrados
            resultado = self.chain.run(question=query, input_documents=docs)
            logging.info("Consulta de QA ejecutada con éxito.")
            return resultado
        except Exception as e:
            logging.error(f"Error al ejecutar la consulta de QA: {e}")
            return None

In [12]:
# Creación y uso de ProcesadorQA
procesador_qa = ProcesadorQA(buscador)
resultado_qa = procesador_qa.ejecutar_qa(query="Haz un resumen de las preguntas y respuestas.")

if resultado_qa:
    print("Resultado de la consulta QA:", resultado_qa)
else:
    print("No se pudo ejecutar la consulta de QA.")

2024-05-11 22:57:24,517 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-11 22:57:27,675 - INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
2024-05-11 22:57:27,679 - INFO - Consulta de QA ejecutada con éxito.


Resultado de la consulta QA: 
1. ¿Cuál es el título de la película sobre la vida de Barack Obama, protagonizada por Devon Terrell? 
Respuesta: Opción A: Barry

2. ¿En qué película original de Netflix, dirigida por Ava DuVernay, se narra la historia de cinco jóvenes acusados injustamente de un crimen en Central Park? 
Respuesta: Opción A: When They See Us 

3. ¿Cuál es el título de la película protagonizada por Sandra Bullock, donde un grupo de personas debe sobrevivir con los ojos vendados para no ser afectados por una fuerza misteriosa que hace que las personas se suiciden? 
Respuesta: Opción A: Bird Box

4. ¿Quién dirigió la película de ciencia ficción 'Aniquilación', protagonizada por Natalie Portman? 
Respuesta: Opción C: Alex Garland

5. ¿Qué actor interpreta al personaje principal en la película 'El irlandés', dirigida por Martin Scorsese? 
Respuesta: Opción A: Robert De Niro

6. ¿Cuál es el título de la serie sobre un misterioso pueblo en Oreg


In [13]:
# Otra consulta de ejemplo

resultado_qa_2 = procesador_qa.ejecutar_qa(query="Nombra algunas de las series de netflix que se mencionan en el json.")
if resultado_qa_2:
    print("Resultado de la consulta QA:", resultado_qa_2)
else:
    print("No se pudo ejecutar la consulta de QA.")

2024-05-11 22:58:07,012 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-11 22:58:09,043 - INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
2024-05-11 22:58:09,047 - INFO - Consulta de QA ejecutada con éxito.


Resultado de la consulta QA:  Algunas de las series de Netflix mencionadas en el json son: Annihilation, Imperial Dreams, The Ritual, Stranger Things, The Office, Dark, Making a Murderer, Sex and the City, The OA, Shameless, The Witcher, Inception, The Crown, Peaky Blinders, Stranger Things, House of Cards, Making a Murderer, The Get Down, Black Mirror, entre otras.


In [14]:
import json

class ProcesadorTrivia:
    def __init__(self, datos_trivia, procesador_qa):
        self.trivia = datos_trivia
        self.procesador_qa = procesador_qa

    def ejecutar_trivia(self):
        """Ejecuta las preguntas de trivia y almacena las respuestas utilizando un Procesador QA."""
        respuestas = {}
        for pregunta in self.trivia:
            pregunta_texto = f"{pregunta['titulo']} Opción A: {pregunta['opcionA']} Opción B: {pregunta['opcionB']} " \
                             f"Opción C: {pregunta['opcionC']} Opción D: {pregunta['opcionD']}"
            # Simular la recuperación de respuesta utilizando ProcesadorQA
            respuesta = self.procesador_qa.ejecutar_qa(query=pregunta_texto)
            respuesta_correcta = pregunta[f"opcion{pregunta['correcta'].upper()}"]
            respuestas[pregunta["id"]] = {
                "pregunta": pregunta_texto,
                "respuesta_obtenida": respuesta,
                "respuesta_correcta": respuesta_correcta
            }
        return respuestas
    
    def guardar_respuestas(self, respuestas, ruta_archivo):
        """Guarda las respuestas de la trivia en un archivo JSON."""
        try:
            with open(ruta_archivo, "w") as f:
                json.dump(respuestas, f, indent=4)
            logging.info(f"Respuestas de la trivia guardadas en '{ruta_archivo}'.")
        except Exception as e:
            logging.error(f"Error al guardar respuestas de la trivia: {e}")

In [ ]:
datos_trivia = procesador_json.cargar_json()
procesador_trivia = ProcesadorTrivia(datos_trivia, procesador_qa)
respuestas_trivia = procesador_trivia.ejecutar_trivia()
procesador_trivia.guardar_respuestas(respuestas_trivia, "respuestas_trivia.json")